In [1]:
from pyspark.sql import SparkSession
import os
import shutil

# Limpa warehouse Iceberg (evita warnings de metadata antigos)
shutil.rmtree("/tmp/iceberg", ignore_errors=True)
os.makedirs("/tmp/iceberg", exist_ok=True)

# SparkSession OTIMIZADA para ICEBERG LOCAL (sem lock-impl para evitar erro de construtor)
spark = SparkSession.builder \
    .appName("Spark-IPS-Iceberg-Local-Fixed-NoLock") \
    .config("spark.jars.packages", 
            "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2") \
    .config("spark.sql.extensions", 
            "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.iceberg_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg_catalog.type", "hadoop") \
    .config("spark.sql.catalog.iceberg_catalog.warehouse", "file:///tmp/iceberg") \
    .config("spark.sql.catalog.iceberg_catalog.io-impl", "org.apache.iceberg.hadoop.HadoopFileIO") \
    .config("spark.sql.catalog.iceberg_catalog.default-spec", "v2") \
    .config("spark.driver.memory", "2g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .enableHiveSupport() \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")  # WARN para ver downloads (mude para "ERROR" após sucesso)

print("=== Spark Iniciado com ICEBERG LOCAL (Sem Lock/AWS)! ===")
print(f"Versão do Spark: {spark.version}")

# Verificação CRÍTICA: Confirma io-impl (deve ser HadoopFileIO)
try:
    io_impl = spark.conf.get("spark.sql.catalog.iceberg_catalog.io-impl")
    print(f"Config io-impl ATUAL: {io_impl}")
    if "HadoopFileIO" in io_impl:
        print("✅ IO local confirmado (HadoopFileIO - sem S3/AWS)!")
    else:
        print("⚠️ IO-impl suspeito - verifique se é HadoopFileIO.")
except Exception as e:
    print(f"Erro ao ler io-impl: {e}")

# Verificação de Catálogos (deve incluir iceberg_catalog após init)
try:
    catalogs = [c.name for c in spark.catalog.listCatalogs()]
    print(f"Catálogos disponíveis: {catalogs} (deve incluir 'iceberg_catalog')")
    if "iceberg_catalog" in catalogs:
        print("✅ Catálogo Iceberg registrado com sucesso!")
    else:
        print("⚠️ 'iceberg_catalog' não listado ainda - teste CREATE para ativar.")
except Exception as e:
    print(f"Erro ao listar catálogos: {e}")

# Testa o catálogo Iceberg (DROP, CREATE simples - sem lock-impl)
try:
    # Teste 1: DROP TABLE teste
    spark.sql("DROP TABLE IF EXISTS iceberg_catalog.default.iceberg_ips")
    print("Teste 1: DROP TABLE OK! (Catálogo acessível).")

    # Teste 2: CREATE simples (Iceberg table)
    spark.sql("CREATE TABLE IF NOT EXISTS iceberg_catalog.default.test_table (id INT) USING iceberg")
    print("Teste 2: CREATE TABLE OK! (Tabela Iceberg criada).")

    # Teste 3: DROP teste
    spark.sql("DROP TABLE IF EXISTS iceberg_catalog.default.test_table")
    print("Teste 3: DROP TABLE teste OK! (Iceberg funcional - pronto para ETL).")

    # Teste 4: Confirma listagem de tabelas vazia
    tables_df = spark.sql("SHOW TABLES IN iceberg_catalog.default")
    print(f"Teste 4: Tabelas em iceberg_catalog.default: {tables_df.count()} (deve ser 0 após drop).")
    
except Exception as e:
    print(f"❌ Erro em um dos testes Iceberg: {e}")
    print("DETALHES: Se mencionar 'LockManager', confirme remoção da config. Reinicie kernel se persistir.")
    print("Dica: Rode manualmente: spark.sql('SHOW CATALOGS').show() para debug.")
else:
    print("\n🎉 TODOS OS TESTES ICEBERG PASSARAM! Catálogo local pronto para DDL/INSERT/UPDATE.")

25/10/02 19:22:05 WARN Utils: Your hostname, DESKTOP-1P6TETU resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/10/02 19:22:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/nice_correia/.cache/pypoetry/virtualenvs/trabalho-spark-RjY8yXlH-py3.12/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/nice_correia/.ivy2/cache
The jars for the packages stored in: /home/nice_correia/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2d514e93-c184-40f6-b973-1964669db6f0;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.2 in central
:: resolution report :: resolve 226ms :: artifacts dl 7ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: o

=== Spark Iniciado com ICEBERG LOCAL (Sem Lock/AWS)! ===
Versão do Spark: 3.5.7
Config io-impl ATUAL: org.apache.iceberg.hadoop.HadoopFileIO
✅ IO local confirmado (HadoopFileIO - sem S3/AWS)!


Catálogos disponíveis: ['spark_catalog'] (deve incluir 'iceberg_catalog')
⚠️ 'iceberg_catalog' não listado ainda - teste CREATE para ativar.
Teste 1: DROP TABLE OK! (Catálogo acessível).
Teste 2: CREATE TABLE OK! (Tabela Iceberg criada).
Teste 3: DROP TABLE teste OK! (Iceberg funcional - pronto para ETL).
Teste 4: Tabelas em iceberg_catalog.default: 0 (deve ser 0 após drop).

🎉 TODOS OS TESTES ICEBERG PASSARAM! Catálogo local pronto para DDL/INSERT/UPDATE.


In [2]:
import os

# Detecta o caminho do CSV dinamicamente para notebook em /notebooks/
# (Sobe 1 nível de notebooks/ para root do projeto: trabalho_spark/)
notebook_dir = os.getcwd()  # Diretório atual (provavelmente /home/nice_correia/trabalho_spark/notebooks)
project_root = os.path.dirname(notebook_dir)  # Sobe 1 nível: notebooks/ -> trabalho_spark/
raw_path = os.path.join(project_root, "data", "ips_brasil.csv")

# Verificação: Mostra caminhos calculados e confirma existência
print(f"Diretório do notebook: {notebook_dir}")
print(f"Root do projeto detectado: {project_root}")
print(f"Caminho absoluto do CSV: {raw_path}")
print(f"Arquivo existe? {os.path.exists(raw_path)}")

# Se não existir (fallback), tenta caminho relativo direto ou absoluto manual
if not os.path.exists(raw_path):
    # Fallback 1: Se notebook foi aberto de outro lugar, tenta relativo ao cwd
    alt_path1 = os.path.join(os.getcwd(), "data", "raw", "ips_brasil.csv")
    print(f"Fallback 1: {alt_path1} (existe? {os.path.exists(alt_path1)})")
    if os.path.exists(alt_path1):
        raw_path = alt_path1
    else:
        # Fallback 2: Caminho absoluto manual (ajuste se o seu home for diferente)
        raw_path = "/home/nice_correia/trabalho_spark/data/raw/ips_brasil.csv"
        print(f"Fallback 2 (manual): {raw_path} (existe? {os.path.exists(raw_path)})")
        if not os.path.exists(raw_path):
            raise FileNotFoundError(f"CSV não encontrado! Verifique se está em {project_root}/data/raw/ips_brasil.csv")

# Carrega o DF com o caminho correto
df = spark.read.option("header", "true").option("inferSchema", "true").csv(raw_path)
print(f"\nDados IPS full carregados de '{raw_path}': {df.count()} linhas, {len(df.columns)} colunas.")

# Verificação rápida: Schema e top 5 linhas chave (com nomes especiais escapados)
print("\nSchema das colunas chave:")
df.select("`Código IBGE`", "`Município`", "`UF`", "`Índice de Progresso Social`").printSchema()
df.select("`Código IBGE`", "`Município`", "`UF`", "`Índice de Progresso Social`").show(5, truncate=False)

# Opcional: Lista todas as colunas para confirmar (79 no total)
print(f"\nTotal de colunas: {len(df.columns)}")
print("Primeiras 10 colunas:", df.columns[:10])

Diretório do notebook: /home/nice_correia/trabalho_spark/notebooks
Root do projeto detectado: /home/nice_correia/trabalho_spark
Caminho absoluto do CSV: /home/nice_correia/trabalho_spark/data/ips_brasil.csv
Arquivo existe? True



Dados IPS full carregados de '/home/nice_correia/trabalho_spark/data/ips_brasil.csv': 5570 linhas, 79 colunas.

Schema das colunas chave:
root
 |-- Código IBGE: integer (nullable = true)
 |-- Município: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- Índice de Progresso Social: double (nullable = true)

+-----------+--------------------------+---+--------------------------+
|Código IBGE|Município                 |UF |Índice de Progresso Social|
+-----------+--------------------------+---+--------------------------+
|1100015    |Alta Floresta D'Oeste (RO)|RO |50.94710852687823         |
|1100023    |Ariquemes (RO)            |RO |55.97475391330499         |
|1100031    |Cabixi (RO)               |RO |51.36453973053614         |
|1100049    |Cacoal (RO)               |RO |61.84526595721548         |
|1100056    |Cerejeiras (RO)           |RO |58.70878800673873         |
+-----------+--------------------------+---+--------------------------+
only showing top 5 rows


Tota

In [3]:
# === Iceberg: DDL (Criação da Tabela) ===
print("\n=== Iceberg: DDL ===")
table_name = "iceberg_catalog.default.iceberg_ips"

# Limpa se existir
spark.sql(f"DROP TABLE IF EXISTS {table_name}")

# Cria tabela Iceberg a partir do DF
df.write.format("iceberg").mode("overwrite").saveAsTable(table_name)

print("Tabela Iceberg criada com sucesso! (79 colunas com nomes especiais suportados).")
print(f"Total de linhas na tabela: {spark.sql(f'SELECT COUNT(*) FROM {table_name}').collect()[0][0]}")

# Verificação: Top 5 por IPS
spark.sql(f"SELECT `Código IBGE`, `Município`, `UF`, `Índice de Progresso Social` FROM {table_name} ORDER BY `Índice de Progresso Social` DESC LIMIT 5").show(truncate=False)


=== Iceberg: DDL ===


25/10/02 20:00:15 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/10/02 20:00:18 WARN HadoopTableOperations: Error reading version hint file file:/tmp/iceberg/default/iceberg_ips/metadata/version-hint.text
java.io.FileNotFoundException: File file:/tmp/iceberg/default/iceberg_ips/metadata/version-hint.text does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:779)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1100)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:769)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.hadoop.fs.ChecksumFileSystem$ChecksumFSInputChecker.<init>(ChecksumFileSystem.java:160)
	at org.apache.hadoop.fs.ChecksumFileSystem.open(ChecksumFileSystem.java:372)
	at 

Tabela Iceberg criada com sucesso! (79 colunas com nomes especiais suportados).
Total de linhas na tabela: 5570


[Stage 13:>                                                         (0 + 1) / 1]

+-----------+-------------------+---+--------------------------+
|Código IBGE|Município          |UF |Índice de Progresso Social|
+-----------+-------------------+---+--------------------------+
|3516853    |Gavião Peixoto (SP)|SP |74.49282395600983         |
|5300108    |Brasília (DF)      |DF |71.25189747327438         |
|3548906    |São Carlos (SP)    |SP |70.96059545595044         |
|5208707    |Goiânia (GO)       |GO |70.49282747376537         |
|3533601    |Nuporanga (SP)     |SP |70.4719550872065          |
+-----------+-------------------+---+--------------------------+



In [4]:
# === Iceberg: INSERT (Adição de Dados) - VERSÃO FINAL CORRIGIDA ===
print("\n=== Iceberg: INSERT ===")
table_name = "iceberg_catalog.default.iceberg_ips"

# Verificação inicial: Confirma colunas chave (baseado no seu df.columns)
print(f"Total de colunas no DF original: {len(df.columns)}")
key_columns = ["Código IBGE", "Município", "UF", "Índice de Progresso Social", "População 2022"]
for col in key_columns:
    exists = col in df.columns
    print(f"Coluna '{col}' existe? {exists}")
    if not exists:
        print(f"AVISO: Coluna '{col}' não encontrada! Colunas similares: {[c for c in df.columns if col.lower() in c.lower()]}")

# Cria DataFrame fictício com schema EXATO do original
from pyspark.sql import Row
data_dict = {col: None for col in df.columns}  # 79 chaves, todas NULL

# Define só colunas que existem (evita erros)
if "Código IBGE" in df.columns:
    data_dict["Código IBGE"] = 9999999  # Fictício (int)
if "Município" in df.columns:
    data_dict["Município"] = "Exemplo Fictício"  # String
if "UF" in df.columns:
    data_dict["UF"] = "XX"  # String
if "Índice de Progresso Social" in df.columns:
    data_dict["Índice de Progresso Social"] = 99.9  # Double alto para topo
if "População 2022" in df.columns:
    data_dict["População 2022"] = 100000  # Int/long fictício (usa coluna real do seu dataset)

# Confirma: Deve ter exatamente 79 chaves
print(f"Chaves no data_dict: {len(data_dict)} (deve ser {len(df.columns)})")

# Cria Row e DataFrame com SCHEMA EXPLÍCITO (resolve inferência com NULLs)
fictitious_row = Row(**data_dict)
fictitious_df = spark.createDataFrame([fictitious_row], df.schema)  # <- CHAVE: Usa df.schema para tipos exatos
print(f"DataFrame fictício criado: {fictitious_df.count()} linha(s), {len(fictitious_df.columns)} colunas.")
print("Schema fictício (primeiras 5 colunas):")
fictitious_df.printSchema()  # Mostra tipos inferidos/forçados (ex: double para IPS)

# INSERT via append (adiciona ao catálogo Iceberg - snapshot novo criado)
fictitious_df.write.format("iceberg").mode("append").saveAsTable(table_name)  # Use saveAsTable para compatibilidade, ou insertInto

print("INSERT: Linha fictícia adicionada com sucesso! (Nova snapshot criada no Iceberg).")

# Verificação: Top 5 linhas ordenadas por IPS (deve incluir a nova no topo)
spark.sql(f"SELECT `Código IBGE`, `Município`, `UF`, `Índice de Progresso Social` FROM {table_name} ORDER BY `Índice de Progresso Social` DESC LIMIT 5").show(truncate=False)

# Conta total (deve ser original +1 = ~5566)
total_rows = spark.sql(f'SELECT COUNT(*) FROM {table_name}').collect()[0][0]
print(f"Total de linhas após INSERT: {total_rows}")

# Verificação extra: Mostra a linha fictícia inserida (com População 2022)
spark.sql(f"SELECT `Código IBGE`, `Município`, `UF`, `Índice de Progresso Social`, `População 2022` FROM {table_name} WHERE `Código IBGE` = 9999999").show(truncate=False)


=== Iceberg: INSERT ===
Total de colunas no DF original: 79
Coluna 'Código IBGE' existe? True
Coluna 'Município' existe? True
Coluna 'UF' existe? True
Coluna 'Índice de Progresso Social' existe? True
Coluna 'População 2022' existe? True
Chaves no data_dict: 79 (deve ser 79)


DataFrame fictício criado: 1 linha(s), 79 colunas.
Schema fictício (primeiras 5 colunas):
root
 |-- Código IBGE: integer (nullable = true)
 |-- Município: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- Área (km²): double (nullable = true)
 |-- População 2022: integer (nullable = true)
 |-- PIB per capita 2021: double (nullable = true)
 |-- Índice de Progresso Social: double (nullable = true)
 |-- Necessidades Humanas Básicas: double (nullable = true)
 |-- Fundamentos do Bem-estar: double (nullable = true)
 |-- Oportunidades: double (nullable = true)
 |-- Nutrição e Cuidados Médicos Básicos: double (nullable = true)
 |-- Água e Saneamento: double (nullable = true)
 |-- Moradia: double (nullable = true)
 |-- Segurança Pessoal: double (nullable = true)
 |-- Acesso ao Conhecimento Básico: double (nullable = true)
 |-- Acesso à Informação e Comunicação: double (nullable = true)
 |-- Saúde e Bem-estar: double (nullable = true)
 |-- Qualidade do Meio Ambiente: double (nullabl

INSERT: Linha fictícia adicionada com sucesso! (Nova snapshot criada no Iceberg).
+-----------+-------------------+---+--------------------------+
|Código IBGE|Município          |UF |Índice de Progresso Social|
+-----------+-------------------+---+--------------------------+
|9999999    |Exemplo Fictício   |XX |99.9                      |
|3516853    |Gavião Peixoto (SP)|SP |74.49282395600983         |
|5300108    |Brasília (DF)      |DF |71.25189747327438         |
|3548906    |São Carlos (SP)    |SP |70.96059545595044         |
|5208707    |Goiânia (GO)       |GO |70.49282747376537         |
+-----------+-------------------+---+--------------------------+

Total de linhas após INSERT: 5571
+-----------+----------------+---+--------------------------+--------------+
|Código IBGE|Município       |UF |Índice de Progresso Social|População 2022|
+-----------+----------------+---+--------------------------+--------------+
|9999999    |Exemplo Fictício|XX |99.9                      |100000

In [5]:
# Verificação de snapshot após INSERT
snapshots_df = spark.sql(f"SELECT snapshot_id, operation, committed_at FROM {table_name}.snapshots ORDER BY committed_at DESC LIMIT 2")
snapshots_df.show(truncate=False)

+-------------------+---------+-----------------------+
|snapshot_id        |operation|committed_at           |
+-------------------+---------+-----------------------+
|1260496121539592792|append   |2025-10-02 20:00:39.085|
|3870061069435861093|overwrite|2025-10-02 20:00:18.681|
+-------------------+---------+-----------------------+



In [6]:
# === Iceberg: UPDATE (Atualização de Dados) - VERSÃO CORRIGIDA ===
print("\n=== Iceberg: UPDATE ===")
table_name = "iceberg_catalog.default.iceberg_ips"

# Verificação: Confirma se Código IBGE 3550308 (São Paulo) existe no dataset
print("Verificando Código IBGE de São Paulo...")
saopaulo_check = spark.sql(f"SELECT `Código IBGE`, `Município`, `UF`, `Índice de Progresso Social` FROM {table_name} WHERE `Município` LIKE '%São Paulo%' LIMIT 1")
saopaulo_check.show(truncate=False)
if saopaulo_check.count() == 0:
    print("AVISO: São Paulo não encontrado! Use outro município. Exemplo: Pegue o primeiro da tabela.")
    # Fallback: Use o primeiro município da tabela como exemplo
    first_city = spark.sql(f"SELECT `Código IBGE`, `Município`, `UF` FROM {table_name} LIMIT 1").collect()[0]
    codigo_ibge = first_city[0]
    municipio = first_city[1]
    uf = first_city[2]
    print(f"Fallback: Usando {municipio} (Código {codigo_ibge}) para UPDATE.")
else:
    row = saopaulo_check.collect()[0]
    codigo_ibge = row[0]
    municipio = row[1]
    uf = row[2]
    print(f"São Paulo encontrado: Código {codigo_ibge}, {municipio}, {uf}")

# DataFrame para UPDATE: Usa o código real encontrado e atualiza IPS para 70.5 (fictício)
update_data = [(codigo_ibge, municipio, uf, 70.5)]  # Novo IPS mais alto

# Schema programático com StructType (resolve acentos - sem string DDL)
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
update_schema = StructType([
    StructField("Código IBGE", IntegerType(), True),  # int, nullable
    StructField("Município", StringType(), True),     # string
    StructField("UF", StringType(), True),            # string
    StructField("Índice de Progresso Social", DoubleType(), True)  # double (com acento OK!)
])
update_df = spark.createDataFrame(update_data, update_schema)
print("DataFrame de UPDATE criado com sucesso (schema programático):")
update_df.show(truncate=False)  # Mostra a linha de update
print(f"Schema: {len(update_df.columns)} colunas")

# Cria temp view para o MERGE
update_df.createOrReplaceTempView("update_temp")

# UPDATE via MERGE INTO (ACID do Iceberg - atualiza baseado em Código IBGE)
spark.sql(f"""
    MERGE INTO {table_name} AS target
    USING update_temp AS source
    ON target.`Código IBGE` = source.`Código IBGE`
    WHEN MATCHED THEN 
        UPDATE SET 
            target.`Índice de Progresso Social` = source.`Índice de Progresso Social`,
            target.`Município` = source.`Município`,
            target.`UF` = source.`UF`
""")

print(f"UPDATE: IPS de {municipio} atualizado para 70.5! (MERGE ACID executado - nova snapshot criada).")

# Verificação: Mostra a linha atualizada (deve ter IPS = 70.5)
spark.sql(f"SELECT `Código IBGE`, `Município`, `UF`, `Índice de Progresso Social` FROM {table_name} WHERE `Código IBGE` = {codigo_ibge}").show(truncate=False)

# Top 5 atualizado (o município deve subir no ranking)
spark.sql(f"SELECT `Código IBGE`, `Município`, `UF`, `Índice de Progresso Social` FROM {table_name} ORDER BY `Índice de Progresso Social` DESC LIMIT 5").show(truncate=False)

# Conta total (deve ser o mesmo, só atualizado)
total_rows = spark.sql(f'SELECT COUNT(*) FROM {table_name}').collect()[0][0]
print(f"Total de linhas após UPDATE: {total_rows} (mesmo total, só atualizado)")


=== Iceberg: UPDATE ===
Verificando Código IBGE de São Paulo...
+-----------+--------------------------+---+--------------------------+
|Código IBGE|Município                 |UF |Índice de Progresso Social|
+-----------+--------------------------+---+--------------------------+
|1303908    |São Paulo de Olivença (AM)|AM |48.6949936570276          |
+-----------+--------------------------+---+--------------------------+

São Paulo encontrado: Código 1303908, São Paulo de Olivença (AM), AM
DataFrame de UPDATE criado com sucesso (schema programático):
+-----------+--------------------------+---+--------------------------+
|Código IBGE|Município                 |UF |Índice de Progresso Social|
+-----------+--------------------------+---+--------------------------+
|1303908    |São Paulo de Olivença (AM)|AM |70.5                      |
+-----------+--------------------------+---+--------------------------+

Schema: 4 colunas


UPDATE: IPS de São Paulo de Olivença (AM) atualizado para 70.5! (MERGE ACID executado - nova snapshot criada).
+-----------+--------------------------+---+--------------------------+
|Código IBGE|Município                 |UF |Índice de Progresso Social|
+-----------+--------------------------+---+--------------------------+
|1303908    |São Paulo de Olivença (AM)|AM |70.5                      |
+-----------+--------------------------+---+--------------------------+

+-----------+--------------------------+---+--------------------------+
|Código IBGE|Município                 |UF |Índice de Progresso Social|
+-----------+--------------------------+---+--------------------------+
|9999999    |Exemplo Fictício          |XX |99.9                      |
|3516853    |Gavião Peixoto (SP)       |SP |74.49282395600983         |
|5300108    |Brasília (DF)             |DF |71.25189747327438         |
|3548906    |São Carlos (SP)           |SP |70.96059545595044         |
|1303908    |São Paulo d

In [7]:
# === Iceberg: Time Travel (Snapshots e Histórico) - VERSÃO CORRIGIDA ===
print("\n=== Iceberg: Time Travel ===")
table_name = "iceberg_catalog.default.iceberg_ips"

# Lista TODOS os snapshots (histórico de mudanças - deve ter 3+: DDL, INSERT, UPDATE)
snapshots_df = spark.sql(f"SELECT * FROM {table_name}.snapshots ORDER BY committed_at DESC")
#print(f"Total de snapshots encontrados: {snapshots_df.count()} (esperado: 3+ após DDL/INSERT/UPDATE)")
#print("Snapshots recentes (histórico completo):")
snapshots_df.show(truncate=False, n=10)  # Mostra todas as colunas e linhas (útil para debug)

# Select customizado com NOMES CORRETOS de colunas (parent_id, não parent_snapshot_id)
#print("\nSnapshots selecionados (colunas chave):")
snapshots_df.select("snapshot_id", "parent_id", "operation", "manifest_list", "summary").show(truncate=False, n=5)

# Time Travel: Consulta versão ANTES do INSERT/UPDATE (snapshot mais antigo, se existir múltiplos)
if snapshots_df.count() > 1:
    # Pega o snapshot mais antigo (última linha após ORDER BY DESC)
    snapshots_list = snapshots_df.collect()  # Coleta todos (poucos snapshots, OK)
    old_snapshot_id = snapshots_list[-1][1]  # snapshot_id é a 2ª coluna (índice 1: committed_at=0, snapshot_id=1)
    old_operation = snapshots_list[-1][3] if len(snapshots_list[-1]) > 3 else "unknown"  # operation
    print(f"\nVersão ANTES das mudanças (snapshot mais antigo ID {old_snapshot_id}, operação: {old_operation}):")
    spark.sql(f"SELECT `Código IBGE`, `Município`, `UF`, `Índice de Progresso Social` FROM {table_name} VERSION AS OF {old_snapshot_id} ORDER BY `Índice de Progresso Social` DESC LIMIT 3").show(truncate=False)
    old_count = spark.sql(f"SELECT COUNT(*) FROM {table_name} VERSION AS OF {old_snapshot_id}").collect()[0][0]
    print(f"Linhas na versão antiga: {old_count} (sem INSERT/UPDATE - só dados originais do DDL)")
else:
    print("\nApenas 1 snapshot (só DDL) - pulando Time Travel antigo. Rode mais DMLs para histórico.")

# Time Travel: Versão ATUAL (após todos os DML - deve incluir INSERT fictício e UPDATE em São Paulo)
print(f"\nVersão ATUAL (snapshot mais recente, após DDL/INSERT/UPDATE):")
current_snapshot_id = snapshots_df.collect()[0][1]  # Primeiro da lista (mais recente)
spark.sql(f"SELECT `Código IBGE`, `Município`, `UF`, `Índice de Progresso Social` FROM {table_name} VERSION AS OF {current_snapshot_id} ORDER BY `Índice de Progresso Social` DESC LIMIT 3").show(truncate=False)
current_count = spark.sql(f'SELECT COUNT(*) FROM {table_name}').collect()[0][0]
print(f"Linhas na versão atual: {current_count} (com INSERT + UPDATE aplicado)")

# Diferença: Compara contagens (deve mostrar impacto do INSERT)
if snapshots_df.count() > 1:
    diff = current_count - old_count
    print(f"Diferença de linhas (atual vs. antigo): +{diff} (devido ao INSERT; UPDATE não altera contagem)")

# Opcional: Rollback para snapshot anterior (ex: desfaz UPDATE/INSERT - COMENTADO por segurança)
# if old_snapshot_id:
#     spark.sql(f"CALL iceberg_system.rollback_to_snapshot('{table_name}', {old_snapshot_id})")
#     print(f"Rollback executado para snapshot {old_snapshot_id}! (Desfaz mudanças - verifique com SELECT COUNT(*))")

print("\nTime Travel completo! Iceberg mantém histórico imutável para auditoria e rollback.")


=== Iceberg: Time Travel ===
+-----------------------+-------------------+-------------------+---------+-------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id          |operation|manifest_list                                                                                                      |summary                                                                                                                                                                                                      

In [8]:
# === Iceberg: DELETE (Remoção de Dados) ===
print("\n=== Iceberg: DELETE ===")
table_name = "iceberg_catalog.default.iceberg_ips"

# DELETE via SQL (remove a linha fictícia INSERTed e, opcionalmente, outra condição)
spark.sql(f"DELETE FROM {table_name} WHERE `Código IBGE` = 9999999")  # Remove a fictícia
# Opcional: spark.sql(f"DELETE FROM {table_name} WHERE `UF` = 'XX'")  # Remove todas de UF fictícia

print("DELETE: Linha fictícia removida! (DELETE ACID executado - nova snapshot).")

# Verificação: Busca a linha deletada (deve retornar vazio)
spark.sql(f"SELECT `Código IBGE`, `Município`, `UF`, `Índice de Progresso Social` FROM {table_name} WHERE `Código IBGE` = 9999999").show(truncate=False)

# Top 5 atualizado (fictícia sumiu, São Paulo ainda no topo)
spark.sql(f"SELECT `Código IBGE`, `Município`, `UF`, `Índice de Progresso Social` FROM {table_name} ORDER BY `Índice de Progresso Social` DESC LIMIT 5").show(truncate=False)

total_rows = spark.sql(f'SELECT COUNT(*) FROM {table_name}').collect()[0][0]
print(f"Total de linhas após DELETE: {total_rows} (original -1 = ~5565)")


=== Iceberg: DELETE ===
DELETE: Linha fictícia removida! (DELETE ACID executado - nova snapshot).
+-----------+---------+---+--------------------------+
|Código IBGE|Município|UF |Índice de Progresso Social|
+-----------+---------+---+--------------------------+
+-----------+---------+---+--------------------------+

+-----------+--------------------------+---+--------------------------+
|Código IBGE|Município                 |UF |Índice de Progresso Social|
+-----------+--------------------------+---+--------------------------+
|3516853    |Gavião Peixoto (SP)       |SP |74.49282395600983         |
|5300108    |Brasília (DF)             |DF |71.25189747327438         |
|3548906    |São Carlos (SP)           |SP |70.96059545595044         |
|1303908    |São Paulo de Olivença (AM)|AM |70.5                      |
|5208707    |Goiânia (GO)              |GO |70.49282747376537         |
+-----------+--------------------------+---+--------------------------+

Total de linhas após DELETE: 55

In [12]:
# === Iceberg: Time Travel (Completo: Antes/Depois de INSERT, UPDATE e DELETE) ===
print("\n=== Iceberg: Time Travel ===")
table_name = "iceberg_catalog.default.iceberg_ips"

# Lista TODOS os snapshots (histórico de mudanças - ORDER BY DESC para mais recente no topo)
snapshots_df = spark.sql(f"SELECT snapshot_id, parent_id, committed_at, operation, summary, manifest_list FROM {table_name}.snapshots ORDER BY committed_at DESC")
print(f"Total de snapshots encontrados: {snapshots_df.count()} (esperado: 4+ após DDL/INSERT/UPDATE/DELETE)")
print("Snapshots recentes (histórico completo - operation e summary chave):")
snapshots_df.show(truncate=False, n=10)  # Mostra colunas chave (útil para debug)

if snapshots_df.count() < 2:
    print("⚠️ Poucos snapshots - rode mais DMLs (INSERT/UPDATE/DELETE) para demo completa.")
else:
    # Coleta snapshots para indexação (poucos, OK para .collect())
    snapshots_list = snapshots_df.collect()
    
    # IDs chave (baseado em ordem DESC: [0]=mais recente (após DELETE), [-1]=mais antigo (inicial))
    current_snapshot_id = snapshots_list[0][0]  # snapshot_id da posição 0 (coluna 0)
    v1_insert_id = snapshots_list[1][0] if len(snapshots_list) > 1 else None  # Após INSERT
    v2_update_id = snapshots_list[2][0] if len(snapshots_list) > 2 else None  # Após UPDATE (antes DELETE)
    old_snapshot_id = snapshots_list[-1][0]  # Mais antigo (inicial, DDL)
    
    # Função auxiliar para query em snapshot específico (foco em fictícia + São Paulo)
    def query_snapshot(snapshot_id, limit=2):
        return spark.sql(f"""
            SELECT `Código IBGE`, `Município`, `UF`, `Índice de Progresso Social` 
            FROM {table_name} VERSION AS OF {snapshot_id} 
            WHERE `Código IBGE` IN (9999999, 3550308)  -- Fictícia + São Paulo (ajuste 3550308 se outro)
            ORDER BY `Índice de Progresso Social` DESC
        """)

    # 1. Versão Inicial (mais antigo: Após DDL, antes INSERT/UPDATE/DELETE)
    print(f"\n1. Versão INICIAL (snapshot {old_snapshot_id} - após DDL, dados originais):")
    try:
        initial_df = query_snapshot(old_snapshot_id)
        initial_df.show(truncate=False)
        initial_count = spark.sql(f"SELECT COUNT(*) FROM {table_name} VERSION AS OF {old_snapshot_id}").collect()[0][0]
        print(f"   Linhas totais: {initial_count} (CSV original)")
        print("   Observação: Sem fictícia (9999999); IPS original de São Paulo (~66)")
    except Exception as e:
        print(f"   Erro na query inicial: {e} (snapshot pode não existir)")

    # 2. Após INSERT (com fictícia adicionada, antes UPDATE/DELETE)
    if v1_insert_id:
        print(f"\n2. Após INSERT (snapshot {v1_insert_id} - fictícia adicionada):")
        try:
            insert_df = query_snapshot(v1_insert_id)
            insert_df.show(truncate=False)
            insert_count = spark.sql(f"SELECT COUNT(*) FROM {table_name} VERSION AS OF {v1_insert_id}").collect()[0][0]
            print(f"   Linhas totais: {insert_count} (+1 da fictícia 9999999 com IPS 99.9 no topo)")
        except Exception as e:
            print(f"   Erro na query INSERT: {e}")
    else:
        print("\n2. Pulando após INSERT (snapshot não encontrado)")

    # 3. Após UPDATE, Antes DELETE (fictícia ainda presente, São Paulo atualizado)
    if v2_update_id:
        print(f"\n3. Após UPDATE, ANTES DELETE (snapshot {v2_update_id} - UPDATE aplicado):")
        try:
            update_df = query_snapshot(v2_update_id)
            update_df.show(truncate=False)
            update_count = spark.sql(f"SELECT COUNT(*) FROM {table_name} VERSION AS OF {v2_update_id}").collect()[0][0]
            print(f"   Linhas totais: {update_count} (mesmo que após INSERT; IPS de São Paulo = 70.5)")
            print("   Observação: Fictícia (9999999) ainda existe; São Paulo atualizado")
        except Exception as e:
            print(f"   Erro na query UPDATE: {e}")
    else:
        print("\n3. Pulando após UPDATE (snapshot não encontrado)")

    # 4. Após DELETE (atual: fictícia removida, UPDATE preservado)
    print(f"\n4. Após DELETE (snapshot {current_snapshot_id} - atual, fictícia removida):")
    try:
        delete_df = query_snapshot(current_snapshot_id)
        delete_df.show(truncate=False)
        current_count = spark.sql(f'SELECT COUNT(*) FROM {table_name}').collect()[0][0]
        print(f"   Linhas totais: {current_count} (-1 da fictícia; volta ao inicial)")
        print("   Observação: Fictícia (9999999) sumiu; São Paulo mantém IPS 70.5 (UPDATE intacto)")
    except Exception as e:
        print(f"   Erro na query DELETE: {e}")

    # Resumo de Mudanças (se snapshots suficientes)
    if len(snapshots_list) >= 4:
        print(f"\nResumo de Mudanças:")
        print(f"   Inicial (snapshot {old_snapshot_id}): {initial_count} linhas")
        print(f"   Após INSERT (snapshot {v1_insert_id}): {insert_count} linhas (+1)")
        print(f"   Após UPDATE (snapshot {v2_update_id}): {update_count} linhas (sem mudança)")
        print(f"   Após DELETE (snapshot {current_snapshot_id}): {current_count} linhas (-1)")

# Opcional: Rollback para snapshot antes do DELETE (ex: v2 - desfaz DELETE, fictícia volta - CUIDADO!)
# if v2_update_id:
#     spark.sql(f"CALL iceberg_system.rollback_to_snapshot('{table_name}', {v2_update_id})")
#     print(f"Rollback executado para snapshot {v2_update_id}! Fictícia voltou - COUNT(*): {spark.sql(f'SELECT COUNT(*) FROM {table_name}').collect()[0][0]}")
#     # Re-execute DELETE para fixar se quiser

print("\nTime Travel Iceberg completo! Histórico imutável - INSERT/UPDATE/DELETE auditáveis em snapshots específicos.")


=== Iceberg: Time Travel ===
Total de snapshots encontrados: 4 (esperado: 4+ após DDL/INSERT/UPDATE/DELETE)
Snapshots recentes (histórico completo - operation e summary chave):
+-------------------+-------------------+-----------------------+---------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------+
|snapshot_id        |parent_id          |committed_at           |operation|summary                                                                                                                                                                      